# 0 Imports

In [1]:
# Gerais
import os
import nltk
import re
import json
import pandas as pd
from datetime import datetime
from time import time as now
start_time = now()

## 0.1 Imports: Buscar novos boatos

In [2]:
# Crawlers
from Modulos.Crawlers.BoatosCrawler import BoatosCrawler
# Rotular os boatos
from Modulos.RotularBoatos import Rotulador

# 1 Utils

## 1.1 Configuração

In [3]:
# Ler Base de dados
BASE_DE_DADOS_ATUAL = "../base_simples/boatos_br_corpus_simples.json"
base_carregada = pd.read_json(BASE_DE_DADOS_ATUAL)
# Urls para procurar
URLS_INICIAIS = [
    "https://www.boatos.org/",
]
ULTIMA_DATA_ATUAL_BASE = datetime.fromisoformat(base_carregada["data-publicacao"][0])
# Path salvar bases
BASE_SIMPLES_PATH = "../base_simples/boatos_br_corpus_simples_att.json"
BASE_COMPLETA_PATH = "../base_completa/boatos_br_corpus_att.json"
BASE_PROCESSADA_PATH = "../base_processada/boatos_br_corpus_processada_att.json"

## 1.2 NLTK

In [4]:
# StopWords
nltk.download('stopwords')
stopwords_portugues = set(nltk.corpus.stopwords.words('portuguese'))
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\harri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\harri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\harri\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# 2 Buscar novos boatos

## 2.2 Crawler

### 2.2.1 Configurando Crawler

In [5]:
crawlerBoatos = BoatosCrawler(urlsIniciais=URLS_INICIAIS)

### 2.2.2 Buscando novos boatos
! Busca apenas boatos que tem data superior a data de publicação do boato mais recente da base

In [6]:
novosBoatos = []
# Buscar noticias novas -> Que não estão na base
novosBoatos = crawlerBoatos.scrape(dateToStop=ULTIMA_DATA_ATUAL_BASE)

Pagina:  https://www.boatos.org/
Tempo gasto:  17.27s
Pagina:  https://www.boatos.org/?paged=2
Tempo gasto:  21.17s
Pagina:  https://www.boatos.org/?paged=3
Tempo gasto:  16.61s
Pagina:  https://www.boatos.org/?paged=4
Tempo gasto:  20.97s
Pagina:  https://www.boatos.org/?paged=5
Parando Busca...
Tempo gasto:  16.82s


## 2.3 Normalizando novos textos

### Limpeza dos textos

#### Lógica de limpeza
* Selecionar apenas letras (remover pontuação, números e outros caracteres)
* Transformar palavras em minusculo
* Remover stopwords
* Manter emojis

In [7]:
def buscaPalavras(texto:str) -> str:
    palavrasMinusculas = re.findall(r'\b[A-zÀ-úü]+\b', texto.lower())
    textoLimpo = [w for w in palavrasMinusculas if w not in stopwords_portugues]
    return " ".join(textoLimpo)

### Rotulação dos boatos encontrados

#### Lógica da rotulação
* Analisar se no texto possui alguma das frases presentes nos dicionários criados

> ./Modulos/RotularBoatos/DicionarioFrasesCategoria.py

In [8]:
CATEGORIASBOATOSFALSO = [
    "Fake news",
    "fake news",
    "Boato sem comprovação",
    "Golpe", 
    "Enganoso", 
    "Exagerado", 
    "é falso",
    "é falsa",
    "não é verdade",
    "não procede",
    "não há provas"
]
CATEGORIASBOATOSVERDADE = [
    "Verdadeiro", 
    "Real com erros",
    "É fato",
]

## 2.4 Construindo o objeto dos novos textos

### Atributos do textos na base BoatosBrData
- **url**: URL de onde foi obtido o texto.
- **data-publicacao**: Data em que foi publicada a checagem do texto.
- **origem**:  De qual site foi retirado
- **categorias**: Categorias do boato, por exemplo: Política, Saúde, Mundo, entre outras.
- **texto**: Texto original do boato que esta circulando na internet
- **texto-normalizado**: Texto normalizado e limpo.
- **rotulo**: Atributo alvo da previsão, podendo ser Verdadeiro ou Falso.

In [9]:
from Class.boatosBrData import BoatosBrData

### 2.4.1 Transformando novos boatos em BoatosBrData

In [10]:
dados_normalizados = []
# Transformação dos boatos encontrados nas páginas
for boato in novosBoatos:
    
    new_boatosBrData = BoatosBrData(
        url= boato['url'],
        dataPublicacao= boato['data-publicacao'],
        origem='boatosorg',
        categorias= boato['categorias'],
        texto= boato['referencia'],
        textoNormalizado= buscaPalavras(boato['referencia']),
        rotulo= Rotulador.AplicarRotulo(boato)
    )
    boatosBrData_dict = new_boatosBrData.getData()
    
    # Informar quais noticias devem ser rotuladas manualmente
    if(boatosBrData_dict['rotulo'] == "NDA"):
        print("[ROTULAR] Não foi possível rotular: ", boatosBrData_dict['url'])
    dados_normalizados.append(boatosBrData_dict)

## 2.5 Limpar boatos vazios da nova base

In [11]:
# Transformar em DataFrame e remover da base, boatos que estão com dados faltando.
novaBaseDF = pd.DataFrame(dados_normalizados)
novaBaseDF= novaBaseDF[novaBaseDF['texto'] != '']
novaBaseDF.dropna()
novaBaseDict = novaBaseDF.to_dict('records')

## 2.6 Salvar base com dados simples

In [12]:
with open(BASE_SIMPLES_PATH,'w+',encoding="utf8") as f:
    json.dump(novaBaseDict,f,indent=3,allow_nan=True,ensure_ascii = False)
    f.close()

# 3 Complementando a base de dados

## 3.1 Extrair novas informações

### 3.1.1 Novas informações extraídas do texto
- Quantidade de emojis
- Quantidade de palavras
- Quantidade de palavras em maiusculo 
- Quantidade de verbos

In [13]:
from Modulos.ComplementarBoatos.Complementador import getNumeroEmojis, getNumeroPalavras, getNumeroPalavrasMaiusculo, getNumeroVerbos

### 3.1.2 Extraindo novas informações

In [14]:
for data in novaBaseDict:
    data['num-emojis'] = getNumeroEmojis(data['texto'])
    data['num-verbos'] = getNumeroVerbos(data['texto'])
    data['num-palavras'] = getNumeroPalavras(data['texto'])
    data['num-palavras-maiusculas'] = getNumeroPalavrasMaiusculo(data['texto'])

## 3.2 Salvar base completa

In [15]:
with open(BASE_COMPLETA_PATH,'w+',encoding="utf8") as f:
    json.dump(novaBaseDict,f,indent=3,allow_nan=True,ensure_ascii = False)
    f.close()

# 4 Pre processar a base de dados

## 4.2 Analisar base pelos dicionários de informações

In [16]:
def loadJson(path:str):
    aux = []
    with open(path,'r') as f:
        aux = json.load(f)
        f.close()
    return aux

def normalizarDadoProcessado(dado:str,listaNormalizacao:dict) -> int:
    return listaNormalizacao[dado] if listaNormalizacao.get(dado) != None else -1

### 4.2.1 Categorias na base

In [17]:
categoriasDict = loadJson("./Processados/categorias/categorias_processadas.json")

### 4.2.2 Origens na base

In [18]:
origensDict = loadJson("./Processados/origem/origens_processadas.json") 

### 4.2.3 Rotulos na base

In [19]:
rotulosDict = {
    'falso': 0,
    'verdade': 1,
    'meia-verdade': 2
}

## 4.3 Pre processar base completa

### 4.3.1 Lógica de pre processamento
- Transformar os seguintes campos da base em valores numéricos:
    - rotulo
    - data-publicacao
    - origem
    - categorias

### 4.3.2 Pre processando os textos da base

In [20]:
for data in novaBaseDict:
    data['categorias'] = [normalizarDadoProcessado(x,categoriasDict) for x in data['categorias']]
    data['origem'] = normalizarDadoProcessado(data['origem'],origensDict)
    data['data-publicacao'] = pd.to_datetime(data['data-publicacao']).timestamp()
    data['rotulo'] = normalizarDadoProcessado(data['rotulo'],rotulosDict)

## 4.4 Salvar base processada

In [21]:
with open(BASE_PROCESSADA_PATH,'w',encoding="utf8") as f:
    json.dump(novaBaseDict,f,indent=2,allow_nan=True,ensure_ascii = False,separators=(',', ':'))
    f.close()

# 5 Nova Base completa

In [22]:
end_time = now()
duration = end_time - start_time
print(f"A duração total de execução foi: {duration} segundos")

paths = [BASE_SIMPLES_PATH,BASE_COMPLETA_PATH,BASE_PROCESSADA_PATH]
for path in paths:
    text = "Erro nao foi salvo"
    if(os.path.exists(path)):
        text = "Salvo com sucesso!"
    print(f"{path} {text}")

A duração total de execução foi: 94.31808018684387 segundos
../base_simples/boatos_br_corpus_simples_att.json Salvo com sucesso!
../base_completa/boatos_br_corpus_att.json Salvo com sucesso!
../base_processada/boatos_br_corpus_processada_att.json Salvo com sucesso!
